<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=21e990a69738273f5df7efd19cf4bbe2042215487d08bf9730216f6cdb555c08
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-22 12:16:19
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  3.11 L
Current:  1.43 C
-------------------
Today PnL: -34.67 K (-0.24%)
Current PnL: -17.07 L (-11.48%)
CY Booked + Current PnL: -5.63 L (-3.79%)
-------------------
Total profit:  2.00 L
Total loss:  -19.07 L
-------------------
Total Booked + Current PnL: 21.35 L (17.54%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.0%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.26 C
Est FTT PnL: 82.59 L (57.75%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,15.30,69.0,H-MC,3.51,199150.0,34480.0,8842.0,1.41,20.94,4.44,26.31,89.0,3.90,1.42,30.59,X40N,NTT,AC
76,TTKPRESTIG,770.00,100.64,48.0,M-SC,3.32,86447.0,-14330.0,14419.0,-0.66,-14.22,16.68,0.09,245.0,-0.99,0.62,13.57,OX40N,NTT,DURABLES
49,NATIONALUM,244.55,-43.66,66.0,H-MC,8.59,112492.0,11988.0,17121.0,0.31,11.93,15.22,28.96,79.0,0.70,0.80,49.95,MH,ATH,METALS
56,RELIANCE,1533.00,-12.54,54.0,H-LC,4.82,219102.0,8736.0,20048.0,-0.21,4.15,9.15,13.68,37.0,0.44,1.57,21.36,XY25,NTT,REFINERIES
41,ITC,452.00,-37.92,47.0,H-LC,2.26,199903.0,-235.0,21130.0,-0.45,-0.12,10.57,10.44,4.0,-0.01,1.43,5.44,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1276.40,81.0,M-SC,6.50,187193.0,6050.0,39423.0,12.28,3.34,21.06,25.10,235.0,0.15,1.34,50.02,XY24,NTT,HEALTHCARE
8,AWL,485.00,-64.07,60.0,H-MC,4.01,255606.0,-46145.0,207578.0,4.16,-15.29,81.21,53.50,116.0,-0.22,1.83,13.53,XY24,NTT,FMCG
31,HINDZINC,730.22,23.40,64.0,M-LC,1.69,201318.0,-3758.0,116322.0,2.15,-1.83,57.78,54.89,52.0,-0.03,1.44,21.82,X5K,ATH,METALS
80,VAIBHAVGBL,521.00,64.19,66.0,H-SC,6.51,142449.0,-40326.0,151395.0,2.11,-22.06,106.28,60.77,125.0,-0.27,1.02,27.59,XR,NTT,JEWELLERY
79,UNITDSPR,1644.00,-11.37,59.0,H-LC,7.62,232441.0,-1703.0,50323.0,1.75,-0.73,21.65,20.77,86.0,-0.03,1.66,6.14,X40N,NTT,BREWERIES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-3.76,52.0,H-LC,1.45,238680.0,-7471.0,86689.0,-3.73,-3.04,36.32,32.18,16.0,-0.09,1.71,31.91,X200,ATH,IT
53,RAJESHEXPO,518.0,1796.46,53.0,L-SC,2.44,51741.0,-85436.0,85528.0,-3.59,-62.28,165.30,0.07,267.0,-1.00,0.37,28.68,OX40N,NTT,JEWELLERY
81,VALIANTORG,838.0,-320.21,49.0,H-SC,6.02,122400.0,-41205.0,162523.0,-2.86,-25.19,132.78,74.15,139.0,-0.25,0.87,56.90,XR,NTT,CHEMICALS
83,VOLTAS,1530.0,-1.08,48.0,H-MC,2.76,207420.0,15678.0,22090.0,-2.61,8.18,10.65,19.69,99.0,0.71,1.48,15.53,XY25,NTT,AC
39,INFY,2275.0,-17.80,49.0,H-LC,8.18,319883.0,6995.0,164676.0,-2.49,2.24,51.48,54.87,3.0,0.04,2.29,9.03,X40,BTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-0.09,57.0,H-MC,1.92,247822.0,91.0,74966.0,-0.14,0.04,30.25,30.3,92.0,0.0,1.77,13.33,X40,ATH,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,94.87,53.0,H-SC,9.32,122742.0,-19737.0,39253.0,-0.72,-13.85,31.98,13.70,163.0,-0.50,0.88,44.87,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,100.64,48.0,M-SC,3.32,86447.0,-14330.0,14419.0,-0.66,-14.22,16.68,0.09,245.0,-0.99,0.62,13.57,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.06,62.0,H-SC,1.27,226665.0,-43002.0,79333.0,-0.55,-15.95,35.00,13.47,138.0,-0.54,1.62,14.64,XY24,NTT,PAINTS
18,CERA,9475.0,-20.76,42.0,H-SC,2.16,144452.0,-31451.0,73468.0,0.26,-17.88,50.86,23.89,149.0,-0.43,1.03,24.75,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-21.19,47.0,H-MC,7.26,106685.0,-25150.0,67372.0,-1.88,-19.08,63.15,32.03,98.0,-0.37,0.76,19.50,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.0,15.3,69.0,H-MC,3.51,199150.0,34480.0,8842.0,1.41,20.94,4.44,26.31,89.0,3.9,1.42,30.59,X40N,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1276.40,81.0,M-SC,6.50,187193.0,6050.0,39423.0,12.28,3.34,21.06,25.10,235.0,0.15,1.34,50.02,XY24,NTT,HEALTHCARE
59,SAIL,228.00,49.16,71.0,M-MC,8.22,235329.0,10367.0,157294.0,0.34,4.61,66.84,74.53,192.0,0.07,1.68,38.35,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,15.30,69.0,H-MC,3.51,199150.0,34480.0,8842.0,1.41,20.94,4.44,26.31,89.0,3.90,1.42,30.59,X40N,NTT,AC
17,CAMS,5211.76,-0.07,63.0,H-SC,3.05,113523.0,11519.0,32411.0,-1.13,11.29,28.55,43.06,122.0,0.36,0.81,30.34,X40N,ATH,MISC
29,HAVELLS,2069.16,-0.09,57.0,H-MC,1.92,247822.0,91.0,74966.0,-0.14,0.04,30.25,30.30,92.0,0.00,1.77,13.33,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,66.0,H-MC,8.59,112492.0,11988.0,17121.0,0.31,11.93,15.22,28.96,79.0,0.70,0.80,49.95,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.66,51.0,M-SC,0.83,100238.0,8740.0,70016.0,-0.10,9.55,69.85,86.07,223.0,0.12,0.72,45.56,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,69.0,H-MC,6.64,186821.0,7013.0,105404.0,0.83,3.90,56.42,62.52,88.0,0.07,1.34,39.32,XR,NTT,BANKS
36,INDIAMART,4850.92,-53.52,32.0,H-SC,11.35,129776.0,6440.0,127323.0,-0.85,5.22,98.11,108.45,119.0,0.05,0.93,28.74,AR,ATH,MISC
85,WIPRO,420.00,-11.52,49.0,M-LC,5.68,155291.0,4346.0,105101.0,-2.11,2.88,67.68,72.51,53.0,0.04,1.11,9.08,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-16.73,41.0,H-LC,6.82,299924.0,-15918.0,128577.0,-0.92,-5.04,42.87,35.67,5.0,-0.12,2.14,8.12,X40N,ATH,FMCG
30,HCLTECH,1943.91,-0.65,43.0,H-LC,9.81,227336.0,-14588.0,77863.0,-1.32,-6.03,34.25,26.15,8.0,-0.19,1.62,7.43,X40,ATH,IT
41,ITC,452.00,-37.92,47.0,H-LC,2.26,199903.0,-235.0,21130.0,-0.45,-0.12,10.57,10.44,4.0,-0.01,1.43,5.44,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-12.93,48.0,H-LC,7.76,219718.0,-32050.0,85251.0,0.61,-12.73,38.80,21.13,27.0,-0.38,1.57,18.19,X40,ATH,PAINTS
39,INFY,2275.00,-17.80,49.0,H-LC,8.18,319883.0,6995.0,164676.0,-2.49,2.24,51.48,54.87,3.0,0.04,2.29,9.03,X40,BTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-3.76,52.0,H-LC,1.45,238680.0,-7471.0,86689.0,-3.73,-3.04,36.32,32.18,16.0,-0.09,1.71,31.91,X200,ATH,IT
41,ITC,452.0,-37.92,47.0,H-LC,2.26,199903.0,-235.0,21130.0,-0.45,-0.12,10.57,10.44,4.0,-0.01,1.43,5.44,X40,NTT,FMCG
64,SIEMENS,4671.5,2.31,60.0,H-LC,3.56,164470.0,-21625.0,69110.0,-1.10,-11.62,42.02,25.51,15.0,-0.31,1.18,21.20,AR,ATH,ELECTRICAL
50,NESTLEIND,1377.0,-9.11,53.0,H-LC,4.28,279864.0,14438.0,42343.0,0.13,5.44,15.13,21.40,11.0,0.34,2.00,13.17,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.54,54.0,H-LC,4.82,219102.0,8736.0,20048.0,-0.21,4.15,9.15,13.68,37.0,0.44,1.57,21.36,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-37.92,47.0,H-LC,2.26,199903.0,-235.0,21130.0,-0.45,-0.12,10.57,10.44,4.0,-0.01,1.43,5.44,X40,NTT,FMCG
73,TCS,4476.75,-25.15,58.0,H-LC,12.29,297905.0,-48051.0,122916.0,-0.24,-13.89,41.26,21.64,1.0,-0.39,2.13,5.53,X40,ATH,IT
79,UNITDSPR,1644.00,-11.37,59.0,H-LC,7.62,232441.0,-1703.0,50323.0,1.75,-0.73,21.65,20.77,86.0,-0.03,1.66,6.14,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-0.65,43.0,H-LC,9.81,227336.0,-14588.0,77863.0,-1.32,-6.03,34.25,26.15,8.0,-0.19,1.62,7.43,X40,ATH,IT
82,VBL,671.64,-16.73,41.0,H-LC,6.82,299924.0,-15918.0,128577.0,-0.92,-5.04,42.87,35.67,5.0,-0.12,2.14,8.12,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,2.31,60.0,H-LC,3.56,164470.0,-21625.0,69110.0,-1.10,-11.62,42.02,25.51,15.0,-0.31,1.18,21.20,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,47.0,H-LC,2.26,199903.0,-235.0,21130.0,-0.45,-0.12,10.57,10.44,4.0,-0.01,1.43,5.44,X40,NTT,FMCG
19,CIPLA,1795.00,-20.12,49.0,H-LC,5.53,213035.0,8535.0,32871.0,-1.32,4.17,15.43,20.25,10.0,0.26,1.52,13.27,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-12.93,48.0,H-LC,7.76,219718.0,-32050.0,85251.0,0.61,-12.73,38.80,21.13,27.0,-0.38,1.57,18.19,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.54,54.0,H-LC,4.82,219102.0,8736.0,20048.0,-0.21,4.15,9.15,13.68,37.0,0.44,1.57,21.36,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,58.0,H-LC,12.29,297905.0,-48051.0,122916.0,-0.24,-13.89,41.26,21.64,1.0,-0.39,2.13,5.53,X40,ATH,IT
39,INFY,2275.00,-17.80,49.0,H-LC,8.18,319883.0,6995.0,164676.0,-2.49,2.24,51.48,54.87,3.0,0.04,2.29,9.03,X40,BTT,IT
41,ITC,452.00,-37.92,47.0,H-LC,2.26,199903.0,-235.0,21130.0,-0.45,-0.12,10.57,10.44,4.0,-0.01,1.43,5.44,X40,NTT,FMCG
82,VBL,671.64,-16.73,41.0,H-LC,6.82,299924.0,-15918.0,128577.0,-0.92,-5.04,42.87,35.67,5.0,-0.12,2.14,8.12,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,9.39,261134.0,-485.0,119704.0,0.06,-0.19,45.84,45.57,7.0,-0.00,1.87,11.11,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7144.44,69.0,L-SC,5.19,79492.0,-14318.0,90939.0,1.51,-15.26,114.40,81.67,269.0,-0.16,0.57,53.94,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,49.0,L-SC,42.32,83136.0,-30413.0,70466.0,-2.30,-26.78,84.76,35.27,268.0,-0.43,0.59,103.48,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-173.86,68.0,M-SC,3.93,85332.0,-16878.0,110266.0,-0.69,-16.51,129.22,91.37,208.0,-0.15,0.61,38.52,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-4.84,46.0,H-SC,12.32,88528.0,-12435.0,106968.0,-0.92,-12.32,120.83,93.63,148.0,-0.12,0.63,31.86,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,52.0,H-SC,12.62,94470.0,-3510.0,25110.0,-0.14,-3.58,26.58,22.05,152.0,-0.14,0.68,37.32,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1276.40,81.0,M-SC,6.50,187193.0,6050.0,39423.0,12.28,3.34,21.06,25.10,235.0,0.15,1.34,50.02,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-15.81,71.0,H-SC,13.42,225736.0,-45025.0,374970.0,-2.12,-16.63,166.11,121.86,133.0,-0.12,1.61,76.59,AR,ATH,IT
60,SAMMAANCAP,326.00,-173.86,68.0,M-SC,3.93,85332.0,-16878.0,110266.0,-0.69,-16.51,129.22,91.37,208.0,-0.15,0.61,38.52,XY25,NTT,FINANCE
80,VAIBHAVGBL,521.00,64.19,66.0,H-SC,6.51,142449.0,-40326.0,151395.0,2.11,-22.06,106.28,60.77,125.0,-0.27,1.02,27.59,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3775.68,58.0,M-SC,4.94,120738.0,-26630.0,78443.0,-0.49,-18.07,64.97,35.16,236.0,-0.34,0.86,23.32,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.65
1,25,44.22
2,50,75.35


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.88
LC    31.21
MC    23.93
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.97
X40      14.98
X40N     11.63
XR       11.47
AR        8.56
XY25      8.39
OX40N     7.96
X200      1.71
MH        1.67
X5K       1.44
SR        1.24
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.89
H-LC    24.64
H-MC    20.96
M-SC    13.46
M-LC     5.52
M-MC     2.67
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.17,-14.04,76.22
FMCG,12.38,-3.67,40.75
FINANCE,10.22,-14.77,64.85
MISC,7.23,-13.75,76.99
BANKS,6.21,-15.01,75.51
PAINTS,5.73,-15.13,32.29
ELECTRICAL,5.62,-4.92,41.73
INSURANCE,3.74,-5.56,45.01
AC,3.61,4.20,11.84


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3050490.0,22
XR,1306539.0,14
AR,1135448.0,9
X40,752134.0,10
X40N,576146.0,9
OX40N,534564.0,10
XY25,373653.0,6
SR,247170.0,2
X5K,116322.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3611246.0,29
M-SC,1366359.0,17
H-MC,1172308.0,15
H-LC,1081965.0,14
M-LC,378027.0,4
M-MC,303915.0,2
L-SC,246933.0,3
L-MC,60307.0,1
L-LC,37729.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1208502.0      6
           AR         783307.0      5
M-SC       XY24       780789.0      7
H-SC       XR         760807.0      7
H-MC       XY24       540216.0      4
H-LC       X40        471836.0      5
M-MC       XY24       303915.0      2
H-SC       X40N       294161.0      4
           OX40N      254786.0      4
           SR         247170.0      2
H-LC       X40N       211771.0      3
H-MC       X40        208714.0      4
H-LC       AR         188814.0      2
M-SC       AR         163327.0      2
H-MC       XY25       163267.0      2
L-SC       XR         161405.0      2
M-LC       XY24       156604.0      2
M-SC       OX40N      126878.0      4
M-LC       X5K        116322.0      1
M-SC       XR         113515.0      2
           XY25       110266.0      1
H-MC       XR         105404.0      1
M-LC       XR         105101.0      1
H-LC       X200        86689.0      1
L-SC       OX40N       85528.0      1
M-SC       X40         71584.0      1
H-MC       X40N        70214.0      2
           OX40N       67372.0      1
H-SC       MH          62513.0      1
H-LC       XY25        62391.0      2
           XY24        60464.0      1
L-MC       XR          60307.0      1
L-LC       XY25        37729.0      1
H-MC       MH          17121.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
